In [1]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("github")
secret_value_1 = user_secrets.get_secret("wandb")


In [2]:
!pip install -U ultralytics wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 79.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.5 MB/s eta 0:00:00:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 4.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# !pip uninstall torch torchvision
# !pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124

In [4]:
# import git
# !rm -rf "/kaggle/working"
# git.Repo.clone_from(f'https://{secret_value_0}@github.com/Beeditor04/hutech_mushroom.git', '/kaggle/working/github')

In [5]:
import yaml

def load_config(path):
    with open(path, 'r') as f:
        config = yaml.load(f, Loader=yaml.FullLoader)
    return config

# Train code

In [6]:
import sys
import os
import yaml
import torch

from ultralytics import YOLO

import wandb
# from wandb.integration.ultralytics import add_wandb_callback


def load_config(path):
    with open(path, 'r') as f:
        config = yaml.load(f, Loader=yaml.FullLoader)
    return config

# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def trainer(config=None):
    PROJECT = "ppe-yolo"
    NAME = "training"

    settings = wandb.Settings(disable_code=True, disable_git=True, _disable_stats=True)

    print("Initial config from sweep:", config)

    with wandb.init(project=PROJECT, job_type="train", settings=settings) as run:
        final_config = run.config
        print("HERE!!!", dict(final_config))
        
        model_name = final_config.get("model", "yolo11n.pt")
        data_path = final_config.get("dataset", "../data/data-ppe_v4.yaml")
        
        epochs = final_config.get("epochs", 100)
        batch_size = final_config.get("batch_size", 16)
        imgsz = final_config.get("imgsz", 640)

        # hyperparameter
        lr0 = final_config.get("lr0", 0.001)
        optimizer = final_config.get("optimizer", "AdamW")
        momentum = final_config.get("momentum", 0.9)
        cos_lr = final_config.get("cos_lr", False)

        # augmentation
        hsv_v = final_config.get("hsv_v", 0.4)
        translate = final_config.get("translate", 0.1)
        scale = final_config.get("scale", 0.5)
        fliplr = final_config.get("fliplr", 0.5)
        mosaic = final_config.get("mosaic", 1.0)
        mixup = final_config.get("mixup", 0.0)
        cutmix = final_config.get("cutmix", 0.0)

        print(f"Training with:")
        print(f"  Model: {model_name}")
        print(f"  Data: {data_path}")
        print(f"  Epochs: {epochs}")
        print(f"  Batch size: {batch_size}")
        print(f"  Image size: {imgsz}")

        model = YOLO(model_name)
        # add_wandb_callback(model, enable_model_checkpointing=True)

        model.train(
            data=data_path,
            epochs=epochs,
            imgsz=imgsz,
            batch=batch_size,
            lr0=lr0,
            optimizer=optimizer,
            momentum=momentum,
            cos_lr=cos_lr,
            hsv_v=hsv_v,
            translate=translate,
            scale=scale,
            fliplr=fliplr,
            mosaic=mosaic,
            mixup=mixup,
            cutmix=cutmix,
            device=device,
            project=PROJECT,
            name=NAME,
        )


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# Wandb setup

In [7]:
if secret_value_1:
    wandb.login(key=secret_value_1)
else:
    print("WANDB_API_KEY not found in environment. Please set it to your WandB API key.")
wandb.require("core")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nambeo2904 (nambeo2904-ppe) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING `wandb.require('core')` is a no-op as it is now the default behavior.


In [8]:
# ## HYPERPARAMETER
# sweep_configuration = {
#     'method': 'grid',
#     'name': 'model_sweep',
#     'metric': {'goal': 'maximize', 'name': 'metrics/mAP50(B)'},
#     'parameters': 
#     {
#         # fixed value
#         'dataset': {'value': "/kaggle/input/ppe-detection-cs406/data-ppe_v4-kaggle.yaml"},
#         'model': {'value': "/kaggle/input/ppe-detection-cs406/yolo10n_sh17.pt"},
#         'batch_size': {'value': 16},
#         'epochs': {'value': 90},
#         'imgsz': {'value': 640}, 

#         # hyperparameter
#         'lr0': {'values': [0.01, 0.001, 0.0001]},
#         'optimizer': {'values': ["AdamW", "Adam", "SGD"]},
#         'momentum': {'value': 0.9},
#         'cos_lr': {'values': [False, True]},
        
#         # augmentation
#         'hsv_v': {'value': 0.4},
#         'translate': {'value': 0.1},
#         'scale': {'value': 0.5},
#         'fliplr': {'value': 0.5},
#         'mosaic': {'value': 1.0},
#         'mixup': {'value': 0.0},
#         'cutmix': {'value': 0.0}
#     }
# }

In [9]:
## HYPERPARAMETER
sweep_configuration = {
    'method': 'bayes',
    'name': 'model_sweep',
    'metric': {'goal': 'maximize', 'name': 'metrics/mAP50(B)'},
    'parameters': 
    {
        # fixed value
        'dataset': {'value': "/kaggle/input/ppe-detection-cs406/data-ppe_v4-kaggle.yaml"},
        'model': {'value': "/kaggle/input/ppe-detection-cs406/yolo10n_sh17.pt"},
        'batch_size': {'value': 16},
        'epochs': {'value': 70},
        'imgsz': {'value': 640}, 

        # hyperparameter
        'lr0': {'value': 0.01},
        'optimizer': {'value': "auto"},
        'momentum': {'value': 0.9},
        'cos_lr': {'value': False},
        
        # augmentation
        'hsv_v': {'min': 0.4, 'max': 0.8},
        'translate': {'min': 0.1, 'max':0.5},
        'scale': {'min': 0.5, 'max': 1.0},
        'fliplr': {'value': 0.5},
        'mosaic': {'value': 1.0},
        'mixup': {'min': 0.0, 'max': 0.5},
        'cutmix': {'min': 0.0, 'max': 0.5}
    }
}

In [10]:
# import torch
# from torch.serialization import safe_globals
# from models.yolo import DetectionModel

# with safe_globals([DetectionModel]):
#     model = torch.hub.load("ultralytics/yolov5", "yolov5n")

In [11]:

import torch
import ultralytics.nn.tasks as tasks

# Allowlist DetectionModel for safe loading
torch.serialization.add_safe_globals([tasks.DetectionModel])

In [12]:
from ultralytics import YOLO
model = YOLO('/kaggle/input/ppe-detection-cs406/yolo10n_sh17.pt')

In [13]:
import wandb

PROJECT = 'ppe-yolo'

sweep_id = wandb.sweep(sweep=sweep_configuration, project=PROJECT)
print(f"Sweep ID: {sweep_id}")

Create sweep with ID: ottxc5lj
Sweep URL: https://wandb.ai/nambeo2904-ppe/ppe-yolo/sweeps/ottxc5lj
Sweep ID: ottxc5lj


In [14]:
print(f"Sweep ID: {sweep_id}")
wandb.agent(sweep_id, function=trainer, count=32)

Sweep ID: ottxc5lj


wandb: Agent Starting Run: i6u4w0il with config:
wandb: 	batch_size: 16
wandb: 	cos_lr: False
wandb: 	cutmix: 0.2227052968699279
wandb: 	dataset: /kaggle/input/ppe-detection-cs406/data-ppe_v4-kaggle.yaml
wandb: 	epochs: 70
wandb: 	fliplr: 0.5
wandb: 	hsv_v: 0.689432524507108
wandb: 	imgsz: 640
wandb: 	lr0: 0.01
wandb: 	mixup: 0.19272913503691924
wandb: 	model: /kaggle/input/ppe-detection-cs406/yolo10n_sh17.pt
wandb: 	momentum: 0.9
wandb: 	mosaic: 1
wandb: 	optimizer: auto
wandb: 	scale: 0.9411971087566704
wandb: 	translate: 0.297317099370487


Initial config from sweep: None


HERE!!! {'batch_size': 16, 'cos_lr': False, 'cutmix': 0.2227052968699279, 'dataset': '/kaggle/input/ppe-detection-cs406/data-ppe_v4-kaggle.yaml', 'epochs': 70, 'fliplr': 0.5, 'hsv_v': 0.689432524507108, 'imgsz': 640, 'lr0': 0.01, 'mixup': 0.19272913503691924, 'model': '/kaggle/input/ppe-detection-cs406/yolo10n_sh17.pt', 'momentum': 0.9, 'mosaic': 1, 'optimizer': 'auto', 'scale': 0.9411971087566704, 'translate': 0.297317099370487}
Training with:
  Model: /kaggle/input/ppe-detection-cs406/yolo10n_sh17.pt
  Data: /kaggle/input/ppe-detection-cs406/data-ppe_v4-kaggle.yaml
  Epochs: 70
  Batch size: 16
  Image size: 640
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.2227052968699279, data=/kaggle/input/ppe-detection-cs406/data-ppe_v4-kaggle.yaml, degrees=0.0, deterministic=True, device=None, 

Overriding model.yaml nc=17 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1      9856  ultralytics.nn.modules.block.SCDown          [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1     36096  ultralytics

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       6/70      3.69G      3.467      3.302      2.891         55        640: 100%|██████████| 118/118 [00:30<00:00,  3.86it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:02<00:00,  4.71it/s]


                   all        402       2327      0.609      0.573      0.585      0.304

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       7/70       3.7G      3.455      3.214       2.87        153        640:  69%|██████▉   | 82/118 [00:26<00:11,  3.11it/s]
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1251, in _try_get_data
    data = self._data_queue.get(timeout=timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/queue.py", line 179, in get
    raise Empty
_queue.Empty

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/tmp/ipykernel_36/657564274.py", line 64, in trainer
    model.train(
  File "/usr/local/lib/python3.11/dist-packages/ultralytics/engine/model.py", line 799, in train
    self.trainer.train()
  File "/usr/local/lib/python3.11/dist-packages/ultralytics/engine/trainer.py", line 227, in train
    self._do_train(world_size)
  File "/usr/local/lib/python3.11/dist-packages/ultralytics/engine/trainer.py", line 388, in _do_train
    fo

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1251, in _try_get_data
    data = self._data_queue.get(timeout=timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/queue.py", line 179, in get
    raise Empty
_queue.Empty

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/wandb/agents/pyagent.py", line 297, in _run_job
    self._function()
  File "/tmp/ipykernel_36/657564274.py", line 64, in trainer
    model.train(
  File "/usr/local/lib/python3.11/dist-packages/ultralytics/engine/model.py", line 799, in train
    self.trainer.train()
  File "/usr/local/lib/python3.11/dist-packages/ultralytics/engine/trainer.py", line 227, in train
    self._do_train(world_size)
  File "/usr/local/lib/python3.11/dist-packages/ultralytics/engine/trainer.py", line 388, in _do_train
    for i, batch 